<a href="https://colab.research.google.com/github/2303A52158/GENERATIVE-AI_2025/blob/main/Generative_AI_2303A52158_Ass_6_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. (1 ponto) Design a multilayer ANN architecture according to the requirements shown below. Train, test, save (.h5) and deploy the model to predict the housing price using Keras deep
learning library**

**2. (1 ponto) Calculate training and testing error metrics.**

**3. (1 ponto) Build the application by loading the saved ANN model.**

                           Tabela 1: ANN Architecture

                   Layer        Neurons    Activation Function
              Hidden Layer-1     15               relu
              Hidden Layer-2     20               relu
              Hidden Layer-3     25               relu
              Hidden Layer-4     20               relu
              Hidden Layer-5     15               relu

                           Tabela 2: Training Parameters
        loss-function epochs batch-size error-metric Optimizer
            MSE         150       32         MAE       adam

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.models import load_model

file_path = '/content/Housing.csv'
data = pd.read_csv(file_path)

categorical_features = data.select_dtypes(include=['object']).columns

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_data = encoder.fit_transform(data[categorical_features])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_features))

numerical_features = data.drop(columns=categorical_features)
X = pd.concat([numerical_features.drop('price', axis=1), encoded_df], axis=1)
y = data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential()
model.add(Dense(15, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(20, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(loss='mean_squared_error', optimizer=Adam(), metrics=['mean_absolute_error'])

history = model.fit(X_train, y_train, epochs=150, batch_size=32, validation_data=(X_test, y_test))

model.save('housing_price_model.h5')

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

print(f"Training Mean Squared Error: {train_mse}")
print(f"Testing Mean Squared Error: {test_mse}")
print(f"Training Mean Absolute Error: {train_mae}")
print(f"Testing Mean Absolute Error: {test_mae}")

loaded_model = load_model('housing_price_model.h5')

def predict_housing_price(features):
    user_input_df = pd.DataFrame([features[:len(numerical_features.columns) - 1]], columns=numerical_features.columns[1:])

    categorical_input_df = pd.DataFrame([features[len(numerical_features.columns) - 1:]], columns=categorical_features)

    categorical_input_df = categorical_input_df.astype(str)

    encoded_user_input = encoder.transform(categorical_input_df)
    encoded_user_input_df = pd.DataFrame(encoded_user_input, columns=encoder.get_feature_names_out(categorical_features))

    final_input_df = pd.concat([user_input_df, encoded_user_input_df], axis=1)

    scaled_input = scaler.transform(final_input_df)

    prediction = loaded_model.predict(scaled_input)
    return prediction[0][0]

user_input = [8000, 3, 2, 2, 1, 1, 'yes', 'no', 'no', 'yes', 'no', 'furnished']
predicted_price = predict_housing_price(user_input)
print(f"Predicted Housing Price: {predicted_price}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 26365231366144.0000 - mean_absolute_error: 4806383.0000 - val_loss: 30129977819136.0000 - val_mean_absolute_error: 5007535.0000
Epoch 2/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 24912993452032.0000 - mean_absolute_error: 4673449.5000 - val_loss: 30129954750464.0000 - val_mean_absolute_error: 5007532.5000
Epoch 3/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 25717721006080.0000 - mean_absolute_error: 4763619.0000 - val_loss: 30129900224512.0000 - val_mean_absolute_error: 5007528.0000
Epoch 4/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 23215887876096.0000 - mean_absolute_error: 4534890.5000 - val_loss: 30129780686848.0000 - val_mean_absolute_error: 5007518.0000
Epoch 5/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 25084318187520.0000 - mean_absolute_error: 4696142.0000 - val_loss: 30129510154240.0000 - val_mean_absolute_error: 5007494.0000
Epoch 6/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 24267

14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Training Mean Squared Error: 817551704064.0
Testing Mean Squared Error: 1842891718656.0
Training Mean Absolute Error: 674481.4375
Testing Mean Absolute Error: 1030785.125


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
Predicted Housing Price: 4610196.0
